# CGAN

*Conditional GAN*

条件付きGAN。与えた条件に沿った画像を生成する。  
「条件」を表すベクトルをGenerator, Discriminatorの両方に与える。与え方はいろいろある。

MNISTを使って、指定した数字の手書き数字画像を生成する

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets import MNIST
from torchvision import transforms
from IPython.display import display

batch_size = 64
nz = 100
noise_std = 0.7
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## MNIST

In [ ]:
dataset = MNIST(
    root="datasets/",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

sample_x, _ = next(iter(dataloader))
n_classes = len(torch.unique(dataset.targets))
w, h = sample_x.shape[-2:]
image_size = w * h
print("batch shape:", sample_x.shape)
print("width:", w)
print("height:", h)
print("image size:", image_size)
print("num classes:", n_classes)

## Discriminator

入力するベクトルの末尾に条件ベクトルを結合する

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(image_size + n_classes, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        self._eye = torch.eye(n_classes, device=device) # 条件ベクトル生成用の単位行列

    def forward(self, x, labels):
        labels = self._eye[labels] # 条件(ラベル)をone-hotベクトルに
        x = x.view(batch_size, -1) # 画像を1次元に
        x = torch.cat([x, labels], dim=1) # 画像と条件を結合
        y = self.net(x)
        return y

## Generator

今回は、条件の情報はノイズに持たせるので、ここでは何もしない

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            self._linear(nz, 128),
            self._linear(128, 256),
            self._linear(256, 512),
            nn.Linear(512, image_size),
            nn.Sigmoid() # 濃淡を0~1に
        )

    def _linear(self, input_size, output_size):
        return nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.BatchNorm1d(output_size),
            nn.ReLU()
        )

    def forward(self, x):
        x = x.view(-1, nz)
        y = self.net(x)
        y = y.view(-1, 1, w, h) # 784 -> 1x28x28
        return y

## 条件の埋め込み

条件の情報をノイズに埋め込む。対応する部分だけ少し大きな値をとるようにした。

In [ ]:
eye = torch.eye(n_classes, device=device)
def make_noise(labels):
    labels = eye[labels]
    labels = labels.repeat_interleave(nz // n_classes, dim=-1)
    z = torch.normal(0, noise_std, size=(len(labels), nz), device=device)
    z = z + labels
    return z

こんな感じ。0の場合は0~9, 1の場合は10~19, 2の場合は20~29 ... の部分が大きくなっている

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(13, 5))
for label in range(10):
    plt.subplot(2, 5, label+1)
    vec = make_noise(torch.tensor([label])).cpu().ravel()
    plt.plot(vec)
    plt.title(label)
plt.tight_layout()

こう見るとわかり辛いが、移動平均をとるとしっかりと特徴が表れているのが分かる

In [ ]:
import numpy as np

plt.figure(figsize=(13, 5))
for label in range(10):
    plt.subplot(2, 5, label+1)
    vec = make_noise(torch.tensor([label])).cpu().numpy().ravel()
    vec = np.convolve(vec, np.ones(n_classes) / n_classes)
    plt.plot(vec)
    plt.title(label)
plt.tight_layout()

## 学習

In [ ]:
# 画像描画
def write(netG, n_rows=1, size=64):
    n_images = n_rows * n_classes
    z = make_noise(torch.tensor(list(range(n_classes)) * n_rows))
    images = netG(z)
    images = transforms.Resize(size)(images)
    img = torchvision.utils.make_grid(images, n_images // n_rows)
    img = transforms.functional.to_pil_image(img)
    display(img)

# 間違ったラベルの生成
def make_false_labels(labels):
    diff = torch.randint(1, n_classes, size=labels.size(), device=device)
    fake_labels = (labels + diff) % n_classes
    return fake_labels

In [ ]:
fake_labels = torch.zeros(batch_size, 1).to(device)
real_labels = torch.ones(batch_size, 1).to(device)
criterion = nn.BCELoss()

def train(netD, netG, optimD, optimG, n_epochs, write_interval=1):
    # 学習モード
    netD.train()
    netG.train()

    for epoch in range(1, n_epochs+1):
        for X, labels in dataloader:
            X = X.to(device) # 本物の画像
            labels = labels.to(device) # 正しいラベル
            false_labels = make_false_labels(labels) # 間違ったラベル

            # 勾配をリセット
            optimD.zero_grad()
            optimG.zero_grad()

            # Discriminatorの学習
            z = make_noise(labels) # ノイズを生成
            fake = netG(z) # 偽物を生成
            pred_fake = netD(fake, labels) # 偽物を判定
            pred_real_true = netD(X, labels) # 本物&正しいラベルを判定
            pred_real_false = netD(X, false_labels) # 本物&間違ったラベルを判定
            # 誤差を計算
            loss_fake = criterion(pred_fake, fake_labels)
            loss_real_true = criterion(pred_real_true, real_labels)
            loss_real_false = criterion(pred_real_false, fake_labels)
            lossD = loss_fake + loss_real_true + loss_real_false
            lossD.backward() # 逆伝播
            optimD.step() # パラメータ更新

            # Generatorの学習
            fake = netG(z) # 偽物を生成
            pred = netD(fake, labels) # 偽物を判定
            lossG = criterion(pred, real_labels) # 誤差を計算
            lossG.backward() # 逆伝播
            optimG.step() # パラメータ更新

        print(f'{epoch:>3}epoch | lossD: {lossD:.4f}, lossG: {lossG:.4f}')
        if write_interval and epoch % write_interval == 0:
            write(netG)

In [ ]:
netD = Discriminator().to(device)
netG = Generator().to(device)
optimD = optim.Adam(netD.parameters(), lr=0.0002)
optimG = optim.Adam(netG.parameters(), lr=0.0002)
n_epochs = 5

print('初期状態')
write(netG)
train(netD, netG, optimD, optimG, n_epochs)

数値ごとの特徴はとらえられている。ただちょっと形が崩れているので、もう少し学習させてみる。

In [ ]:
train(netD, netG, optimD, optimG, 30, 5)

学習が完了した。最終的に完成したGeneratorで10個ずつ画像を生成してみる。

In [ ]:
write(netG, 10)

まだ形が崩れているが、数字を認識できる程度にはなっている。

## CDCGAN

*Conditional Deep Convolitional GAN*

条件付きDCGAN。GeneratorとDiscriminatorにCNNを使うだけ。画像がきれいになることを期待する。

### Discriminator

畳み込みで得た特徴ベクトルに、ラベル（条件）のone-hotベクトルを結合して、全結合層に入力する。

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            self._conv_layer(1, 16, 4, 2, 1),
            self._conv_layer(16, 32, 4, 2, 1),
            self._conv_layer(32, 64, 3, 2, 0),
            nn.Conv2d(64, 128, 3, 1, 0),
            nn.Flatten()
        )

        self.fc = nn.Sequential(
            nn.Linear(128 + n_classes, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

        self._eye = torch.eye(n_classes, device=device) # 条件ベクトル生成用の単位行列

    def _conv_layer(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x, labels):
        x = self.conv(x) # 特徴抽出
        labels = self._eye[labels] # 条件(ラベル)をone-hotベクトルに
        x = torch.cat([x, labels], dim=1) # 画像と条件を結合
        y = self.fc(x)
        return y

### Generator

先ほど同様、条件の情報はノイズに持たせる。

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            self._convT(nz, 128, 3, 1, 0),
            self._convT(128, 64, 3, 2, 0),
            self._convT(64, 32, 4, 2, 1),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Sigmoid()
        )

    def _convT(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        x = x.view(-1, nz, 1, 1)
        y = self.net(x)
        return y

### 学習

まずは5epoch

In [ ]:
netD = Discriminator().to(device)
netG = Generator().to(device)
optimD = optim.Adam(netD.parameters(), lr=0.0002)
optimG = optim.Adam(netG.parameters(), lr=0.0002)
n_epochs = 5

print('初期状態')
write(netG)
train(netD, netG, optimD, optimG, n_epochs)

線が綺麗になっている。追加でもう20epoch学習させてみる。

In [ ]:
train(netD, netG, optimD, optimG, 30, 10)

In [ ]:
write(netG, 10)

線は綺麗だが、少し形が乱れることがあるよう。

## 生成器に与える条件をいじる

生成器に与えるノイズには条件の情報を持たせているが、その条件を変な風にいじったらどうなるのか。

In [ ]:
def write_from_label(netG, label, n_images=10, size=64):
    labels = torch.tensor([label]*n_images).to(device)
    labels = labels.repeat_interleave(nz // n_classes, dim=-1)
    z = torch.normal(0, noise_std, size=(len(labels), nz), device=device)
    z = z + labels
    images = netG(z)
    images = transforms.Resize(size)(images)
    img = torchvision.utils.make_grid(images, len(z))
    img = transforms.functional.to_pil_image(img)
    display(img)

### 偏りを変化させる

現在は正規分布に従ってノイズを生成した後、生成したい数字に対応する箇所に1を足すことで条件の情報をノイズに持たせている。ではこの**1**を変化させるとどうなるか

まずは大きくしてみる（1 -> 10）

In [ ]:
label = [0, 0, 0, 0, 0, 10, 0, 0, 0, 0]
write_from_label(netG, label)

特徴はしっかり捉えられている。そして乱数の影響が少なくなるので、生成される画像はほぼ同じになる。

次に小さくしてみる（1 -> 0.5）

In [ ]:
label = [0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0]
write_from_label(netG, label)

条件の制約が弱まり、バラバラな画像になった。特徴はぎりぎり捉えられているか。

### 二つの数字を条件に入れる

現在生成したい数字に対応する箇所が大きくなるようにしている。では、複数の個所を大きくしたらどうなるのか。

3と6を大きくしてみよう

In [ ]:
label = [0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
write_from_label(netG, label)

基本的には3または6だが、たまに3と6が混ざったような画像も生成される。

### 何も与えない

ただの乱数をそのまま与えてみる

In [ ]:
label = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
write_from_label(netG, label)

ごちゃごちゃ。乱数によって生まれた偏りによって数字っぽくなることもある様子。

### 全部与える

全部の個所を大きくしてみる

In [ ]:
label = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
write_from_label(netG, label)

おー、って感じ。